In [1]:
%load_ext autotime

time: 0 ns (started: 2022-12-22 04:05:04 +07:00)


In [2]:
import pandas as pd

import numpy as np

import warnings
warnings.filterwarnings("ignore")

time: 0 ns (started: 2022-12-22 04:05:04 +07:00)


# DATASET

In [3]:
df = pd.read_csv("generated_dataset.csv")
df.head(5)

,Wellhead Temp. (C),Wellhead Press (psi),MMCFD- gas,BOPD (barrel of oil produced per day),BWPD (barrel of water produced per day),BSW - basic solid and water (%),CO2 mol. (%) @ 25 C & 1 Atm.,Gas Grav.,CR-corrosion defect
0,53.35,1105.13,12.87,1378.93,2812.62,75.64,3.3628,0.7205,0.2245
1,72.25,1026.31,3.42,1028.75,919.92,44.21,3.8679,0.8940,0.2262
2,65.08,722.96,6.23,2017.92,1212.42,17.55,2.3552,0.7661,0.2305
3,60.71,1557.23,11.71,558.22,1716.09,65.79,1.7253,0.7738,0.2303
4,46.19,1304.42,8.58,1280.47,1929.22,37.45,1.8327,0.7611,0.2202


time: 32 ms (started: 2022-12-22 04:05:04 +07:00)


# TARGET AND FEATURES SPLIT

In [4]:
target = df['CR-corrosion defect']
features = df.drop('CR-corrosion defect', axis=1)

time: 0 ns (started: 2022-12-22 04:05:04 +07:00)


In [5]:
CR_level = []
for row in df['CR-corrosion defect']:
    if row <= 0.211: CR_level.append('Low')
    if row > 0.211: CR_level.append('High')

stratify = pd.DataFrame(CR_level)

time: 0 ns (started: 2022-12-22 04:05:04 +07:00)


# TRAIN TEST SPLIT

In [6]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(features, target, test_size=0.25, random_state=42, stratify=stratify)

time: 265 ms (started: 2022-12-22 04:05:04 +07:00)


# DATA NORMALIZATION

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

time: 16 ms (started: 2022-12-22 04:05:05 +07:00)


In [8]:
col_names = ['Wellhead Temp. (C)', 'Wellhead Press (psi)', 'MMCFD- gas', 'BOPD (barrel of oil produced per day', 'BWPD (barrel of water produced per day', 'BSW - basic solid and water (%)', 'CO2 mol. (%) @ 25 C & 1 Atm.', 'Gas Grav.']

time: 0 ns (started: 2022-12-22 04:05:05 +07:00)


In [9]:
Xtrain_scaled = pd.DataFrame(data = Xtrain_scaled, columns = col_names)
Xtest_scaled = pd.DataFrame(data = Xtest_scaled, columns = col_names)

time: 0 ns (started: 2022-12-22 04:05:05 +07:00)


In [10]:
Xtrain_scaled

,Wellhead Temp. (C),Wellhead Press (psi),MMCFD- gas,BOPD (barrel of oil produced per day,BWPD (barrel of water produced per day,BSW - basic solid and water (%),CO2 mol. (%) @ 25 C & 1 Atm.,Gas Grav.
0,0.950915,0.052967,0.783940,0.727889,0.239421,0.134971,0.548182,0.711957
1,0.087805,0.327101,0.764298,0.804252,0.246465,0.001346,0.683888,0.376359
2,0.402439,0.770111,0.512998,0.750716,0.108114,0.891507,0.520693,0.718750
3,0.313720,0.328202,0.228769,0.714967,0.804792,0.076742,0.290032,0.649457
4,0.775305,0.749275,0.518775,0.433478,0.964242,0.210030,0.062051,0.348279
...,...,...,...,...,...,...,...,...
7714,0.053354,0.188394,0.181976,0.988303,0.376989,0.009985,0.525279,0.610960
7715,0.750000,0.399860,0.211438,0.633808,0.882921,0.097722,0.724942,0.221014
7716,0.308841,0.099724,0.619873,0.317734,0.017887,0.094693,0.522793,0.305707
7717,0.628354,0.376369,0.166378,0.310409,0.566533,0.486144,0.166455,0.784420


time: 16 ms (started: 2022-12-22 04:05:05 +07:00)


# OUTLIER DETECTION - ISOLATION FOREST

In [11]:
from sklearn.ensemble import IsolationForest

isol =  IsolationForest(n_estimators=200 ,contamination = 0.05)

isol.fit(Xtrain_scaled)

IsolationForest(contamination=0.05, n_estimators=200)

time: 750 ms (started: 2022-12-22 04:05:05 +07:00)


In [12]:
Xtrain_predict = isol.predict(Xtrain_scaled)

Xtrain_scaled['Predict'] = Xtrain_predict

Xtrain_scaled['Predict'] = Xtrain_scaled['Predict'].astype('category')

time: 375 ms (started: 2022-12-22 04:05:05 +07:00)


In [13]:
ytrain = pd.DataFrame(data = ytrain, columns = ['CR-corrosion defect'])
ytrain['Predict'] = Xtrain_predict

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [14]:
Xtrain_scaled = Xtrain_scaled[Xtrain_scaled.Predict != -1]
ytrain = ytrain[ytrain.Predict != -1]

Xtrain_scaled = Xtrain_scaled.drop('Predict', axis=1)
ytrain =ytrain.drop('Predict', axis=1)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [15]:
ytrain = np.ravel(ytrain)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


# OUTLIER DETECTION - ONE CLASS SVM

In [16]:
from sklearn.svm import OneClassSVM

ee = OneClassSVM(nu=0.01)
yhat = ee.fit_predict(Xtrain_scaled)

time: 156 ms (started: 2022-12-22 04:05:06 +07:00)


In [17]:
Xtrain_scaled['yhat'] = yhat

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [18]:
ytrain = pd.DataFrame(data = ytrain, columns = ['CR-corrosion defect'])
ytrain['yhat'] = yhat

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [19]:
Xtrain_scaled = Xtrain_scaled[Xtrain_scaled.yhat != -1]
ytrain = ytrain[ytrain.yhat != -1]

Xtrain_scaled = Xtrain_scaled.drop('yhat', axis=1)
ytrain =ytrain.drop('yhat', axis=1)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [20]:
ytrain = np.ravel(ytrain)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [21]:
Xtrain_scaled

,Wellhead Temp. (C),Wellhead Press (psi),MMCFD- gas,BOPD (barrel of oil produced per day,BWPD (barrel of water produced per day,BSW - basic solid and water (%),CO2 mol. (%) @ 25 C & 1 Atm.,Gas Grav.
0,0.950915,0.052967,0.783940,0.727889,0.239421,0.134971,0.548182,0.711957
1,0.087805,0.327101,0.764298,0.804252,0.246465,0.001346,0.683888,0.376359
2,0.402439,0.770111,0.512998,0.750716,0.108114,0.891507,0.520693,0.718750
3,0.313720,0.328202,0.228769,0.714967,0.804792,0.076742,0.290032,0.649457
4,0.775305,0.749275,0.518775,0.433478,0.964242,0.210030,0.062051,0.348279
...,...,...,...,...,...,...,...,...
7714,0.053354,0.188394,0.181976,0.988303,0.376989,0.009985,0.525279,0.610960
7715,0.750000,0.399860,0.211438,0.633808,0.882921,0.097722,0.724942,0.221014
7716,0.308841,0.099724,0.619873,0.317734,0.017887,0.094693,0.522793,0.305707
7717,0.628354,0.376369,0.166378,0.310409,0.566533,0.486144,0.166455,0.784420


time: 15 ms (started: 2022-12-22 04:05:06 +07:00)


# PCA

In [22]:
from sklearn.decomposition import PCA

pca = PCA(.95)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [23]:
pca.fit(Xtrain_scaled)

PCA(n_components=0.95)

time: 109 ms (started: 2022-12-22 04:05:06 +07:00)


In [24]:
Xtrain_scaled = pca.transform(Xtrain_scaled)
Xtest_scaled = pca.transform(Xtest_scaled)

time: 15 ms (started: 2022-12-22 04:05:06 +07:00)


In [25]:
Xtrain_scaled

array([[-0.17104783,  0.35649565, -0.59581518, ..., -0.30015827,
        -0.01194728, -0.05066918],
       [-0.19496312,  0.03544075,  0.03277535, ...,  0.13079242,
        -0.02027461,  0.28514959],
       [ 0.51014456,  0.18111118, -0.00663192, ...,  0.0327648 ,
        -0.18293824,  0.06171103],
       ...,
       [-0.05946758, -0.13195808, -0.03939725, ..., -0.17780039,
         0.37994047, -0.07359206],
       [ 0.11693532,  0.12150788, -0.16814899, ...,  0.053175  ,
         0.41961212, -0.02417697],
       [-0.01232195,  0.12657166, -0.00248695, ...,  0.4391955 ,
         0.49064684, -0.50036315]])

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


# LINEAR REGRESSION (ELASTIC NET)

In [26]:
from sklearn.linear_model import ElasticNet

en = ElasticNet()
en.fit(Xtrain_scaled, ytrain)

ElasticNet()

time: 15 ms (started: 2022-12-22 04:05:06 +07:00)


## K-CROSS VALIDATOR - ELASTIC NET

In [27]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=3, shuffle = True, random_state=125)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [28]:
from sklearn.model_selection import RandomizedSearchCV

param_grid_en = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0], 'l1_ratio': np.arange(0, 1, 0.01)}
grid_en = RandomizedSearchCV(en, param_grid_en, cv=cv, n_iter=200, n_jobs=-1)

time: 0 ns (started: 2022-12-22 04:05:06 +07:00)


In [29]:
grid_en.fit(Xtrain_scaled, ytrain)

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=125, shuffle=True),
                   estimator=ElasticNet(), n_iter=200, n_jobs=-1,
                   param_distributions={'alpha': [1e-05, 0.0001, 0.001, 0.01,
                                                  0.1, 0.0, 1.0, 10.0, 100.0],
                                        'l1_ratio': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0

time: 4.03 s (started: 2022-12-22 04:05:06 +07:00)


In [30]:
alpha = grid_en.best_params_['alpha']
l1_ratio = grid_en.best_params_['l1_ratio']

time: 0 ns (started: 2022-12-22 04:05:10 +07:00)


In [31]:
en_opt = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
en_opt.fit(Xtrain_scaled, ytrain)

ElasticNet(alpha=0.0001, l1_ratio=0.08)

time: 0 ns (started: 2022-12-22 04:05:10 +07:00)


In [32]:
prediction_en = en_opt.predict(Xtest_scaled)

time: 0 ns (started: 2022-12-22 04:05:10 +07:00)


## SCORE - ELASTIC NET

In [33]:
from sklearn import metrics

mae_en = metrics.mean_absolute_error(ytest, prediction_en)
mse_en = metrics.mean_squared_error(ytest, prediction_en)
rmse_en = np.sqrt(metrics.mean_squared_error(ytest, prediction_en))

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction_en))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction_en))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction_en)))

Mean Absolute Error: 0.023310908951676278
Mean Squared Error: 0.0010971032429847529
Root Mean Squared Error: 0.03312254886002514
time: 0 ns (started: 2022-12-22 04:05:10 +07:00)


# NEURAL NETWORK (MLP REGRESSOR)

In [34]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
mlp.fit(Xtrain_scaled, ytrain)

MLPRegressor()

time: 437 ms (started: 2022-12-22 04:05:11 +07:00)


## K-CROSS VALIDATOR - MLP

In [35]:
param_grid_mlp = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                  'activation': ['tanh', 'relu'],
                  'solver': ['sgd', 'adam'],
                  'alpha': [0.0001, 0.05],
                  'learning_rate': ['constant','adaptive']}
grid_mlp = RandomizedSearchCV(mlp, param_grid_mlp, cv=cv, n_iter=200, n_jobs=-1)

time: 0 ns (started: 2022-12-22 04:05:11 +07:00)


In [36]:
grid_mlp.fit(Xtrain_scaled, ytrain)

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=125, shuffle=True),
                   estimator=MLPRegressor(), n_iter=200, n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.0001, 0.05],
                                        'hidden_layer_sizes': [(50, 50, 50),
                                                               (50, 100, 50),
                                                               (100,)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']})

time: 26 s (started: 2022-12-22 04:05:11 +07:00)


In [72]:
hidden_layer_sizes = grid_mlp.best_params_['hidden_layer_sizes']
activation = grid_mlp.best_params_['activation']
solver = grid_mlp.best_params_['solver']
alpha = grid_mlp.best_params_['alpha']
learning_rate = grid_mlp.best_params_['learning_rate']

time: 0 ns (started: 2022-12-22 04:38:03 +07:00)


In [38]:
mlp_opt = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, alpha=alpha, learning_rate=learning_rate)
mlp_opt.fit(Xtrain_scaled, ytrain)

MLPRegressor(hidden_layer_sizes=(50, 100, 50))

time: 750 ms (started: 2022-12-22 04:05:37 +07:00)


In [39]:
prediction_mlp = mlp_opt.predict(Xtest_scaled)

time: 0 ns (started: 2022-12-22 04:05:38 +07:00)


## SCORE - MLP

In [40]:
mae_mlp = metrics.mean_absolute_error(ytest, prediction_mlp)
mse_mlp = metrics.mean_squared_error(ytest, prediction_mlp)
rmse_mlp = np.sqrt(metrics.mean_squared_error(ytest, prediction_mlp))

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction_mlp))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction_mlp))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction_mlp)))

Mean Absolute Error: 0.009773214324702417
Mean Squared Error: 0.00043480768763531896
Root Mean Squared Error: 0.020852042768882837
time: 0 ns (started: 2022-12-22 04:05:38 +07:00)


# RANDOM FOREST

In [41]:
from sklearn.ensemble import RandomForestRegressor

cforest = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=4)

cforest.fit(Xtrain_scaled, ytrain)

RandomForestRegressor(n_estimators=500, n_jobs=4, random_state=42)

time: 4.92 s (started: 2022-12-22 04:05:38 +07:00)


## K-CROSS VALIDATOR - RF

In [42]:
max_features = ['sqrt', 'auto']
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)]
min_samples_leaf=[1,3,4]
n_estimators=[50,20,50,100]
min_samples_split = [2, 6, 10]
bootstrap = [True, False]

param_grid_rf = dict(max_features = max_features, max_depth = max_depth, min_samples_leaf=min_samples_leaf,n_estimators=n_estimators, min_samples_split=min_samples_split, bootstrap=bootstrap)
grid_rf = RandomizedSearchCV(cforest, param_grid_rf, cv = cv, n_iter=200,n_jobs=-1)

time: 0 ns (started: 2022-12-22 04:05:43 +07:00)


In [43]:
grid_rf.fit(Xtrain_scaled, ytrain)

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=125, shuffle=True),
                   estimator=RandomForestRegressor(n_estimators=500, n_jobs=4,
                                                   random_state=42),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['sqrt', 'auto'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [50, 20, 50, 100]})

time: 1min 18s (started: 2022-12-22 04:05:43 +07:00)


In [78]:
n_estimators = grid_rf.best_params_['n_estimators']
min_samples_leaf = grid_rf.best_params_['min_samples_leaf']
max_features = grid_rf.best_params_['max_features']
max_depth = grid_rf.best_params_['max_depth']
min_samples_split = grid_rf.best_params_['min_samples_split']
bootstrap = grid_rf.best_params_['bootstrap']

time: 0 ns (started: 2022-12-22 04:40:14 +07:00)


In [45]:
cf_opt = RandomForestRegressor(max_features = max_features, max_depth = max_depth, min_samples_leaf=min_samples_leaf,n_estimators=n_estimators, min_samples_split=min_samples_split, bootstrap=bootstrap)
cf_opt.fit(Xtrain_scaled, ytrain)

RandomForestRegressor(max_depth=80, min_samples_leaf=3, min_samples_split=6)

time: 2.72 s (started: 2022-12-22 04:07:02 +07:00)


In [46]:
prediction_rf = cf_opt.predict(Xtest_scaled)

time: 31 ms (started: 2022-12-22 04:07:04 +07:00)


## SCORE - RF

In [47]:
mae_rf = metrics.mean_absolute_error(ytest, prediction_rf)
mse_rf = metrics.mean_squared_error(ytest, prediction_rf)
rmse_rf = np.sqrt(metrics.mean_squared_error(ytest, prediction_rf))

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction_rf))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction_rf))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction_rf)))

Mean Absolute Error: 0.011872050716296665
Mean Squared Error: 0.000511242343821799
Root Mean Squared Error: 0.02261066880527418
time: 0 ns (started: 2022-12-22 04:07:05 +07:00)


# DECISION TREE

In [48]:
from sklearn.tree import DecisionTreeRegressor

dtree = DecisionTreeRegressor(max_depth=4,min_samples_split=2,min_samples_leaf=1,max_features=0.25)
dtree.fit(Xtrain_scaled, ytrain)

DecisionTreeRegressor(max_depth=4, max_features=0.25)

time: 0 ns (started: 2022-12-22 04:07:05 +07:00)


## K-CROSS VALIDATOR - DT

In [49]:
param_grid_dt = {"max_depth" : [1,3,5,7,9,11,12],
                 "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
                 "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
                 "max_features":["auto","log2","sqrt",None],
                 "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90]}
grid_dt = RandomizedSearchCV(dtree, param_grid_dt, cv = cv, n_iter=200,n_jobs=-1)

time: 0 ns (started: 2022-12-22 04:07:05 +07:00)


In [50]:
grid_dt.fit(Xtrain_scaled, ytrain)

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=125, shuffle=True),
                   estimator=DecisionTreeRegressor(max_depth=4,
                                                   max_features=0.25),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'max_depth': [1, 3, 5, 7, 9, 11, 12],
                                        'max_features': ['auto', 'log2', 'sqrt',
                                                         None],
                                        'max_leaf_nodes': [None, 10, 20, 30, 40,
                                                           50, 60, 70, 80, 90],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10],
                                        'min_weight_fraction_leaf': [0.1, 0.2,
                                                                     0.3, 0.4,
                                                      

time: 532 ms (started: 2022-12-22 04:07:05 +07:00)


In [51]:
min_samples_leaf = grid_dt.best_params_['min_samples_leaf']
max_features = grid_dt.best_params_['max_features']
max_depth = grid_dt.best_params_['max_depth']
min_weight_fraction_leaf = grid_dt.best_params_['min_weight_fraction_leaf']
max_leaf_nodes = grid_dt.best_params_['max_leaf_nodes']

time: 0 ns (started: 2022-12-22 04:07:05 +07:00)


In [52]:
dt_opt = RandomForestRegressor(min_samples_leaf=min_samples_leaf, max_features=max_features,
                                max_depth=max_depth, min_weight_fraction_leaf=min_weight_fraction_leaf, max_leaf_nodes=max_leaf_nodes)
dt_opt.fit(Xtrain_scaled, ytrain)

RandomForestRegressor(max_depth=11, max_features=None, max_leaf_nodes=40,
                      min_samples_leaf=3, min_weight_fraction_leaf=0.1)

time: 984 ms (started: 2022-12-22 04:07:05 +07:00)


In [53]:
prediction_dt = dt_opt.predict(Xtest_scaled)

time: 16 ms (started: 2022-12-22 04:07:06 +07:00)


## SCORE - DT

In [54]:
mae_dt = metrics.mean_absolute_error(ytest, prediction_dt)
mse_dt = metrics.mean_squared_error(ytest, prediction_dt)
rmse_dt = np.sqrt(metrics.mean_squared_error(ytest, prediction_dt))

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction_dt))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction_dt))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction_dt)))

Mean Absolute Error: 0.0221076378239343
Mean Squared Error: 0.0010055458766292595
Root Mean Squared Error: 0.031710343369778564
time: 0 ns (started: 2022-12-22 04:07:06 +07:00)


# XG BOOST

In [55]:
from sklearn.ensemble import GradientBoostingRegressor

xgb = GradientBoostingRegressor(n_estimators=500,learning_rate=0.05,random_state=100,max_features=0.1)
xgb.fit(Xtrain_scaled,ytrain)

GradientBoostingRegressor(learning_rate=0.05, max_features=0.1,
                          n_estimators=500, random_state=100)

time: 1.17 s (started: 2022-12-22 04:07:06 +07:00)


## K-CROSS VALIDATOR - XG BOOST

In [56]:
from sklearn.model_selection import GridSearchCV

param_grid_xgb = {'n_estimators': np.arange(100, 500, 100), 'learning_rate': [0.01, 0.05, 0.1],
                  'max_depth': [2, 3, 4, 5, 6, 7], 'max_features': [0.1, 0.5, 1]}
grid_xgb = GridSearchCV(xgb, param_grid_xgb, cv=cv, n_jobs=-1)

time: 0 ns (started: 2022-12-22 04:07:07 +07:00)


In [57]:
grid_xgb.fit(Xtrain_scaled, ytrain)

GridSearchCV(cv=KFold(n_splits=3, random_state=125, shuffle=True),
             estimator=GradientBoostingRegressor(learning_rate=0.05,
                                                 max_features=0.1,
                                                 n_estimators=500,
                                                 random_state=100),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [2, 3, 4, 5, 6, 7],
                         'max_features': [0.1, 0.5, 1],
                         'n_estimators': array([100, 200, 300, 400])})

time: 1min 31s (started: 2022-12-22 04:07:07 +07:00)


In [58]:
learning_rate = grid_xgb.best_params_['learning_rate']
max_depth = grid_xgb.best_params_['max_depth']
n_estimators = grid_xgb.best_params_['n_estimators']
max_features = grid_xgb.best_params_['max_features']

time: 0 ns (started: 2022-12-22 04:08:39 +07:00)


In [59]:
xgb_opt = GradientBoostingRegressor(n_estimators=n_estimators,learning_rate=learning_rate,random_state=100,max_features=max_features, max_depth=max_depth)
xgb_opt.fit(Xtrain_scaled, ytrain)

GradientBoostingRegressor(learning_rate=0.05, max_depth=7, max_features=0.5,
                          n_estimators=400, random_state=100)

time: 5.89 s (started: 2022-12-22 04:08:39 +07:00)


In [60]:
prediction_xgb = xgb_opt.predict(Xtest_scaled)

time: 32 ms (started: 2022-12-22 04:08:45 +07:00)


## SCORE - XG BOOST

In [61]:
mae_xgb = metrics.mean_absolute_error(ytest, prediction_xgb)
mse_xgb = metrics.mean_squared_error(ytest, prediction_xgb)
rmse_xgb = np.sqrt(metrics.mean_squared_error(ytest, prediction_xgb))

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction_xgb))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction_xgb))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction_xgb)))

Mean Absolute Error: 0.010061290439431957
Mean Squared Error: 0.00047209819665295873
Root Mean Squared Error: 0.021727820798528294
time: 0 ns (started: 2022-12-22 04:08:45 +07:00)


# K-NEAREST NEIGHBOR

In [62]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors = 5, metric = 'minkowski', p = 2)
knn.fit(Xtrain_scaled, ytrain)

KNeighborsRegressor()

time: 16 ms (started: 2022-12-22 04:08:45 +07:00)


## K-CROSS VALIDATOR - KNN

In [63]:
param_grid_knn = {'n_neighbors': [5, 10, 15, 20, 25], 'p': [1, 2], 'leaf_size': [30, 40, 50]}
grid_knn = GridSearchCV(knn, param_grid_knn, cv=cv, n_jobs=-1)

time: 0 ns (started: 2022-12-22 04:08:45 +07:00)


In [64]:
grid_knn.fit(Xtrain_scaled, ytrain)

GridSearchCV(cv=KFold(n_splits=3, random_state=125, shuffle=True),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'leaf_size': [30, 40, 50],
                         'n_neighbors': [5, 10, 15, 20, 25], 'p': [1, 2]})

time: 1.78 s (started: 2022-12-22 04:08:45 +07:00)


In [82]:
n_neighbors = grid_knn.best_params_['n_neighbors']
p = grid_knn.best_params_['p']
leaf_size = grid_knn.best_params_['leaf_size']

time: 0 ns (started: 2022-12-22 04:43:08 +07:00)


In [66]:
knn_opt = KNeighborsRegressor(n_neighbors=n_neighbors, p=p, leaf_size=leaf_size)
knn_opt.fit(Xtrain_scaled, ytrain)

KNeighborsRegressor(n_neighbors=10)

time: 16 ms (started: 2022-12-22 04:08:47 +07:00)


In [67]:
prediction_knn = knn_opt.predict(Xtest_scaled)

time: 109 ms (started: 2022-12-22 04:08:47 +07:00)


## SCORE - KNN

In [68]:
mae_knn = metrics.mean_absolute_error(ytest, prediction_knn)
mse_knn = metrics.mean_squared_error(ytest, prediction_knn)
rmse_knn = np.sqrt(metrics.mean_squared_error(ytest, prediction_knn))

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction_knn))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction_knn))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction_knn)))

Mean Absolute Error: 0.012918394869801788
Mean Squared Error: 0.0005565176361057131
Root Mean Squared Error: 0.02359062602191203
time: 0 ns (started: 2022-12-22 04:08:47 +07:00)


# DATA OUTPUT

In [69]:
from openpyxl import load_workbook

df2 = pd.read_csv("generated_dataset.csv")

en_features = en_opt.predict(pca.transform(scaler.transform(features)))
mlp_features = mlp_opt.predict(pca.transform(scaler.transform(features)))
rf_features = cf_opt.predict(pca.transform(scaler.transform(features)))
dt_features = dt_opt.predict(pca.transform(scaler.transform(features)))
xgb_features = xgb_opt.predict(pca.transform(scaler.transform(features)))
knn_features = knn_opt.predict(pca.transform(scaler.transform(features)))

df2['EN Prediction'] = en_features
df2['MLP Prediction'] = mlp_features
df2['RF Prediction'] = rf_features
df2['DT Prediction'] = dt_features
df2['XGB Prediction'] = xgb_features
df2['KNN Prediction'] = knn_features

time: 1.31 s (started: 2022-12-22 04:08:47 +07:00)


In [70]:
mae = [mae_en, mae_mlp, mae_rf, mae_dt, mae_xgb, mae_knn]
mse = [mse_en, mse_mlp, mse_rf, mse_dt, mse_xgb, mse_knn]
rmse = [rmse_en, rmse_mlp, rmse_rf, rmse_dt, rmse_xgb, rmse_knn]
regressor = ['EN', 'MLP', 'RF', 'DT', 'XGB', 'KNN']

df3 = pd.DataFrame(list(zip(regressor, mae, mse, rmse)), columns = ['Regressor', 'MAE', 'MSE', 'RMSE'])

time: 0 ns (started: 2022-12-22 04:08:49 +07:00)


In [71]:
ExcelWorkbook = load_workbook('CR Prediction.xlsx')
writer = pd.ExcelWriter('CR Prediction.xlsx', engine = 'openpyxl')
writer.book = ExcelWorkbook

df2.to_excel(writer, sheet_name = 'Regression Approach')
df3.to_excel(writer, sheet_name = 'Regression Score')

writer.close()

time: 6.08 s (started: 2022-12-22 04:08:49 +07:00)
